# Standard evaluators and target functions.
In this notebook we will demonstrate how to use the target functions with the standard evaluators.

In [ ]:
import inspect
import json
import pandas as pd

from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import F1ScoreEvaluator, GroundednessEvaluator

## Target function
Target function is a callable, which should be applied to data before calculating metrics by the evaluator. In this example, we will apply a simple function, answering questions and then apply F1 and Groundedness evaluator to outputs. Due to current limitations function needs to be saved as a separate python file and then imported. The function will generate the answer.

In [ ]:
def fun_response(question):
    data = {"sun": "The star.", "alexander the great": "Binkey"}
    for k, v in data.items():
        if k in question.lower():
            return {"answer": v}
    return {"answer": "I do not know."}


lines = inspect.getsource(fun_response)
with open("get_response.py", "w") as fp:
    fp.write(lines)

from get_response import fun_response as target_fun

target_fun("What is Sun?")

## Data
Because we want to generate the groundedness evaluation, we will need the data context.

In [ ]:
data_raw = pd.DataFrame(
    {
        "question": ["What is the name of Alexander the Great horse?", "What is Sun?"],
        "context": [
            "Bucephalus or Bucephalas was the horse of Alexander the Great, and one of the most famous horses of classical antiquity.",
            "The Sun is the star at the center of the Solar System.",
        ],
        "ground_truth": ["Bucephalus", "The star."],
    }
)
data_raw.to_json("data.jsonl", lines=True, orient="records")

## Configuration
To use groundednessEvaluator, we will need to deploy the model.
Please create the deployment of gpt-3.5 or gpt-4 in Azure Open AI and create the `openai_auth.json` file with the next contents.

```json
{
    "AZURE_OPENAI_API_KEY": super_secret_key",
    "AZURE_OPENAI_ENDPOINT": "https:deployment_namei2.openai.azure.com/"
}
```

In [ ]:
secure_data = json.load(open("openai_auth.json"))

configuration = AzureOpenAIModelConfiguration(
    azure_endpoint=secure_data["AZURE_OPENAI_ENDPOINT"],
    api_key=secure_data["AZURE_OPENAI_API_KEY"],
    api_version="2023-07-01-preview",
    azure_deployment="gpt-35-turbo-1106",  # Please use the name of a model you have deployed.
)

## Run the evaluation

In [ ]:
results = evaluate(
    data="data.jsonl",
    target=target_fun,
    evaluators={
        "f1_evaluator": F1ScoreEvaluator(),
        "groundedess_evaluator": GroundednessEvaluator(model_config=configuration),
    },
)

View the results

In [ ]:
print(f"{results['metrics']=}")
pd.DataFrame(results["rows"])